# Pipeline - School Roads - Roads

The purpose of this notebook is to gather school roads from pre-specified points. These points provide a general location of schools within a particular region. Because this data may come from various sources, the assumption of this notebook is that each row of data provided has a column indicating the latitude and longitude. This point is a relative location for a larger "polygon" that represents the space by which roads may be near.

The data collected by this notebook will later be used to determine whether specific segments of roads are near a school and if they are, to further analyze whether those roads have street-level signage indicating and warning drivers about the school zone. This data in turn is used to help automatically determine the iRAP 5-star attribute code for school signage.

## Imported Data

The assumption is that incoming data will be in parquet format and include at least columns for "lat" (latitude) and "lon" (longitude)

## Exported Data

Data exported from this notebook will be in ".shp" format and will include the linestrings of roads near the schools provided in the source data.

In [2]:
# Parameters cell used to indicate parameters which will be used at runtime.
# Note: the below is a default parameter value which is overridden when the
# notebook is executed as part of a pipeline via Prefect + Papermill

name = "usa"
link = "https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1819.zip"
unzip = True
target = "EDGE_GEOCODE_PUBLICSCH_1819.xlsx"
lat_colname = "LAT"
lon_colname = "LON"

In [14]:
import os
import warnings

import geopandas as gpd
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import shapely
import shapely.ops as sp_ops
from pyproj import Proj, Transformer
from shapely.geometry import LinearRing, LineString

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")
ox.config(log_console=True, log_level=10, use_cache=True)

In [4]:
def buffer_polygon(
    polygon: shapely.geometry.polygon.Polygon, buffer_dist: int = 25
) -> shapely.geometry.polygon.Polygon:
    """
    We don't receive many roads from just the polygon of the school amenity
    so we buffer the polygon for increased area using the same fxnality osmnx does
    with ox.graph_from_polygon.clean_periphery:
    https://github.com/gboeing/osmnx/blob/8653183bb966cfa826e92f1c95b11efa51f99572/osmnx/graph.py#L415

    buffer_dist == amount to buffer the polygon in meters(?)
    """

    poly_proj, crs_utm = ox.projection.project_geometry(polygon)
    poly_proj_buff = poly_proj.buffer(buffer_dist)
    poly_buff, _ = ox.projection.project_geometry(
        poly_proj_buff, crs=crs_utm, to_latlong=True
    )
    return poly_buff

In [5]:
def gather_road_geometry_around_amenity(
    lat: float, lon: float, amenity_types: list
) -> pd.DataFrame:
    # single lat/lon location of the school
    point = tuple([lat, lon])

    # find amenities near point
    poi_df = ox.geometries.geometries_from_point(
        point, tags={"amenity": True}, dist=600
    )

    # limit results to the first amenity with a polygon geometry type
    # there's a better way, but the hope here is that there aren't too many other overlapping polygons
    if (
        len(
            poi_df[
                (poi_df["amenity"].isin(amenity_types))
                & (poi_df["geometry"].geom_type == "Polygon")
            ]
        )
        > 0
    ):
        school_poi = poi_df[
            (poi_df["amenity"].isin(amenity_types))
            & (poi_df["geometry"].geom_type == "Polygon")
        ].iloc[0]
    else:
        return None

    # loop for buffer range of amenity polygon to make sure we receive roads back without errors
    for x in range(25, 80, 3):
        try:
            # buffer/inflate the geometry polygon to encompass nearby roads with next steps
            buffered_school_poi_geometry = buffer_polygon(
                polygon=school_poi["geometry"], buffer_dist=x
            )

            # get roads graph based on buffered polygon
            school_roads_graph = ox.graph_from_polygon(
                polygon=buffered_school_poi_geometry,
                network_type="drive",
                truncate_by_edge=True,
            )

            # if we're able to run without a valueerror being emitted, break loop
            break

        # if we reach a value error, continue the try's until we don't error
        except ValueError:
            # print("Failed with buffered polygon of dist {}, trying again with greater buffer.".format(x))
            continue

    # extract roads pandas edgelist from school roads nx graph
    roads_df = nx.to_pandas_edgelist(school_roads_graph)

    return roads_df

In [6]:
# read in our source school data
df = pd.read_parquet("{}/data/{}_schools.parquet".format(os.getcwd(), name))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NCESSCH     5 non-null      int64  
 1   LEAID       5 non-null      int64  
 2   NAME        5 non-null      object 
 3   OPSTFIPS    5 non-null      int64  
 4   STREET      5 non-null      object 
 5   CITY        5 non-null      object 
 6   STATE       5 non-null      object 
 7   ZIP         5 non-null      int64  
 8   STFIP       5 non-null      int64  
 9   CNTY        5 non-null      int64  
 10  NMCNTY      5 non-null      object 
 11  LOCALE      5 non-null      int64  
 12  lat         5 non-null      float64
 13  lon         5 non-null      float64
 14  CBSA        5 non-null      object 
 15  NMCBSA      5 non-null      object 
 16  CBSATYPE    5 non-null      int64  
 17  CSA         5 non-null      object 
 18  NMCSA       5 non-null      object 
 19  NECTA       5 non-null      objec

In [7]:
# loop through points for schools, gathering a list of dataframes with road geometries around those points
df_list = []
for index, row in df[~(df["lat"].isna()) & ~(df["lon"].isna())].iterrows():

    print("Processing data for: {}".format(row["NAME"]))

    road_points_df = gather_road_geometry_around_amenity(
        lat=row["lat"],
        lon=row["lon"],
        amenity_types=["school", "kindergarten", "preeschool"],
    )

    if type(road_points_df) != None:
        # append the dataframe to a list for later concatenation
        df_list.append(road_points_df)

Processing data for: IYC Chicago
Processing data for: IL Center for Rehab & Educ-R
Processing data for: Horizon Science Acad-McKinley Pk
Processing data for: Horizon Science Acad-Belmont
Processing data for: Betty Shabazz Internl Charter Sch


In [8]:
# merge the results, reset the index and show a sample
merged_df = pd.concat(df_list)
merged_df = merged_df.reset_index()
merged_df.head(10)

,index,source,target,highway,length,name,geometry,oneway,osmid,lanes
0,0,261210541,261285902,residential,200.845,West Monroe Street,"LINESTRING (-87.67906929999999 41.8799731, -87...",False,32671335,NaN
1,1,261285635,261285902,residential,140.514,South Leavitt Street,"LINESTRING (-87.6814593 41.8786708, -87.681461...",False,24102186,NaN
2,2,261285902,261320009,residential,110.600,West Monroe Street,NaN,False,32671335,NaN
3,3,261285902,261285635,residential,140.514,South Leavitt Street,"LINESTRING (-87.6814947 41.8799342, -87.681481...",False,24102186,NaN
4,4,261285902,5970397468,residential,141.460,South Leavitt Street,"LINESTRING (-87.6814947 41.8799342, -87.681507...",False,24102186,NaN
5,5,261285902,261210541,residential,200.845,West Monroe Street,"LINESTRING (-87.6814947 41.8799342, -87.679810...",False,32671335,NaN
6,6,261320009,261285902,residential,110.600,West Monroe Street,NaN,False,32671335,NaN
7,7,5970397468,261285902,residential,141.460,South Leavitt Street,"LINESTRING (-87.6815304 41.8812061, -87.681510...",False,24102186,NaN
8,0,261200837,1308208304,residential,199.401,West Washburne Avenue,"LINESTRING (-87.6737694 41.8658851, -87.674742...",False,115908233,NaN
9,1,261200837,261207380,residential,99.949,South Wolcott Avenue,"LINESTRING (-87.6737694 41.8658851, -87.673769...",False,116725117,NaN


In [50]:
# convert certain columns to strings to avoid list incompatibilities with gpd files
merged_df["osmid"] = merged_df["osmid"].astype(str)
merged_df["name"] = merged_df["name"].astype(str)

In [51]:
# convert to geopandas dataframe for export
gdf = gpd.GeoDataFrame(merged_df, geometry="geometry")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   index     36 non-null     int64   
 1   source    36 non-null     int64   
 2   target    36 non-null     int64   
 3   highway   36 non-null     object  
 4   length    36 non-null     float64 
 5   name      36 non-null     object  
 6   geometry  27 non-null     geometry
 7   oneway    36 non-null     bool    
 8   osmid     36 non-null     object  
 9   lanes     36 non-null     object  
dtypes: bool(1), float64(1), geometry(1), int64(3), object(4)
memory usage: 2.7+ KB


In [52]:
# lists may have been returned for object types, convert to string for gdf export compatibility
gdf["name"] = gdf["name"].astype(str)
gdf["highway"] = gdf["highway"].astype(str)
gdf["osmid"] = gdf["osmid"].astype(str)
gdf["lanes"] = gdf["lanes"].astype(str)

In [53]:
gdf = gdf.dropna(subset=["geometry"]).explode()

In [54]:
# export to shape file
gdf.to_parquet("{}/data/{}_school_roads.parquet".format(os.getcwd(), name))